In [238]:
import pandas as pd 
import numpy as np
import os
import warnings
import mmcv

warnings.filterwarnings('ignore')

In [239]:
def format(df):
    features = ['pelvis', 'chestLower', 'neckLower', 'head', 'lShldrBend', 'lForearmBend', 'lHand', 'rShldrBend', 'rForearmBend', 'rHand', 'lThighBend', 'lShin','lFoot', 'rThighBend', 'rShin', 'rFoot', 'chestUpper', 'lThumb3_end', 'rThumb3_end']

    for feature in features:
        df[feature] = df[f'{feature}_T_glob'].str.cat(df[f'{feature}_T_glob.1'], sep=',').str.cat(df[f'{feature}_T_glob.2'], sep=',')
        
    df['rWrist'] = df['rHand_T_glob'].str.cat(df['rHand_T_glob.1'], sep=',').str.cat(df['rHand_T_glob.2'], sep=',')
    df['lWrist'] = df['lHand_T_glob'].str.cat(df['lHand_T_glob.1'], sep=',').str.cat(df['lHand_T_glob.2'], sep=',')
    df['lAnkle'] = df['lFoot_T_glob'].str.cat(df['lFoot_T_glob.1'], sep=',').str.cat(df['lFoot_T_glob.2'], sep=',')
    df['rAnkle'] = df['rFoot_T_glob'].str.cat(df['rFoot_T_glob.1'], sep=',').str.cat(df['rFoot_T_glob.2'], sep=',')
    df['tipLeftHand'] = df['lThumb3_end_T_glob'].str.cat(df['lThumb3_end_T_glob.1'], sep=',').str.cat(df['lThumb3_end_T_glob.2'], sep=',')
    df['tipRightHand'] = df['rThumb3_end_T_glob'].str.cat(df['rThumb3_end_T_glob.1'], sep=',').str.cat(df['rThumb3_end_T_glob.2'], sep=',')

    features = ['pelvis', 'chestLower', 'neckLower', 'head', 'lShldrBend', 'lForearmBend', 'lWrist', 'lHand', 'rShldrBend', 'rForearmBend', 'rWrist', 'rHand', 'lThighBend', 'lShin','lAnkle','lFoot', 'rThighBend', 'rShin','rAnkle', 'rFoot', 'chestUpper', 'tipLeftHand', 'lThumb3_end', 'tipRightHand', 'rThumb3_end']

    df = df[['Frame', 'Time(s)','pelvis', 'chestLower', 'neckLower', 'head', 'lShldrBend', 'lForearmBend', 'lWrist', 'lHand', 'rShldrBend', 'rForearmBend', 'rWrist', 'rHand', 'lThighBend', 'lShin','lAnkle','lFoot', 'rThighBend', 'rShin','rAnkle', 'rFoot', 'chestUpper', 'tipLeftHand', 'lThumb3_end', 'tipRightHand', 'rThumb3_end']]
    
    for feature in features:
        values = df[feature].values
        for i in range(1, values.shape[0]):
            values[i] = values[i].split(',')
        df[feature] = values


    df.drop([0], axis=0, inplace=True)
    return df

In [240]:
def to_categorical(annotation_file):
    data = pd.read_csv(annotation_file, sep="\t")
    cols = list(data.columns)
    data.head()
    action_class, label = cols[0], cols[1]

    def label_change(x):
        if x == 'Mono':
            return 1
        elif x == 'Bi':
            return 1
        else:
            return 2

    data[label] = data[label].apply(label_change(label))
    data.to_csv(annotation_file, index=False)

In [241]:
def to_txt(csv_file, column_number):
    # Read the CSV file using pandas
    df = pd.read_csv(csv_file)

    # Check if the given column_number is valid
    if column_number < 0 or column_number >= df.shape[1]:
        raise ValueError("Invalid column number")

    # Extract the required column using the column_number
    column_data = df.iloc[:, column_number]

    # Create a txt file with the same name as the CSV file
    txt_file = csv_file.replace(".csv", f"_column{column_number}.txt")

    # Write the column data to the txt file
    with open(txt_file, 'w') as file:
        for value in column_data:
            file.write(str(value) + "\n")

    print(f"Column {column_number} from {csv_file} has been saved in {txt_file}")

In [242]:
def to_skeleton(df, name_df, to_path):
    joints_df = df.iloc[:, 2:]
    data = joints_df.to_numpy()

    num_joints = data.shape[1]
    num_frames = data.shape[0]
    body_info = [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 2]

    with open(to_path+ '{}'.format(name_df) + '.skeleton', 'w') as f:

        f.write("{}\n".format(num_frames))
        
        for frame in data:
            f.write("1\n")
            f.write("{} {} {} {} {} {} {} {} {} {} {}\n".format(body_info[0], body_info[1], body_info[2], body_info[3], body_info[4], body_info[5], body_info[6], body_info[7], body_info[8], body_info[9], body_info[10]))
            f.write("{}\n".format(num_joints))
            for joint in range(num_joints):

                x = float(frame[joint][0])
                y = float(frame[joint][1])
                z = float(frame[joint][2])

                f.write("{} {} {}\n".format(x, y, z))

    f.close()


In [243]:
annotation = pd.read_csv('Dataset/Annotation_gloses.csv', sep='\t') # On lit le fichier d'annotation

from_path = "./Dataset/Cassiopée_Allbones/"
to_path = "./Dataset/Skeletons_3/"

len_path = len([name for name in os.listdir(from_path)])

cols = list(annotation.columns) # On récupère une liste des noms des colonnes
action_class, label = cols[0], cols[1] # On récupère les noms des colonnes

print(f'Converting {len_path} files to skeleton format...')

for file_name in os.listdir(from_path): # On parcourt les fichiers du dataset brut de MocapLab

    num = annotation[annotation[action_class] == file_name[:-4]].index.values # On récupère l'index de la ligne du fichiers d'annotation correspondant au fichier du dataset brut
    if len(num) > 0:
        num = num[0]
        name_df = "S001C001P001A" + "{:03d}".format(num) # On crée le nom du fichier de sortie en fonction de l'index de la ligne
        df = pd.read_csv(from_path + file_name, sep=";", index_col=False)
        df = format(df)
        to_skeleton(df, name_df, to_path) # On convertit le fichier du dataset brut en fichier skeleton

print('Done !')

Converting 169 files to skeleton format...

Done !
